In [2]:
pip install -r requirements_python3_6.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached compound-word-splitter-0.4.tar.gz (2.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached emoji-1.7.0.tar.gz (175 kB)
  Preparing metadata (setup.py) ... done
  Using cached etaprogress-1.1.1.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Using cached gtimer-1.0.0b5.tar.gz (23 kB)
  Preparing metadata (setup.py) ... done
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached jsonpickle-1.5.2-py2.py3-none-any.whl (37 kB)
     |████████████████████████████████| 297 kB 7.0 MB/s            
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
     |████████████████████████████████| 259 kB 13.1 MB/s            
     |████████████████████████████████| 99 kB 13.7 MB/s

In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.1 MB 7.1 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -r requirements_film2.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 112 kB 7.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 130 kB 33.2 MB/s            
     |████████████████████████████████| 920 kB 37.2 MB/s            
     |████████████████████████████████| 840 kB 35.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.6 MB 33.4 MB/s            
     |████████████████████████████████| 4.0 MB 45.9 MB/s            
     |████████████████████████████████| 144 kB 24.8 MB/s            
  Using cached imageio-2.6.0-py3-none-any.whl (3.3 MB)
     |████████████████████████████████| 26.9 MB 28.3 MB/s            
     |████████████████████████████████| 309 kB 20.0 MB/s            
     |████████████████████████████████| 74 kB 5.0 MB/s             
     |████████████████████████████████| 150.5 MB 342 kB/s            
  Installing build depende

In [12]:
pip install --upgrade pip --user

  Using cached https://files.pythonhosted.org/packages/a4/6d/6463d49a933f547439d6b5b98b46af8742cc03ae83543e4d7688c2420f8b/pip-21.3.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
def get_reordered_actionSeq(list_of_list_of_actions, seen_objects, task_type, language_goal):
    receptacles = {'ArmChair', 'BathtubBasin', 'Bed', 'Cabinet', 'Cart', 'CoffeeMachine', 'CoffeeTable', 
                'CounterTop', 'Desk', 'DiningTable', 'Drawer', 'Dresser', 'Fridge', 'GarbageCan', 
                'Microwave', 'Ottoman', 'Safe', 'Shelf', 'SideTable', 'SinkBasin', 'Sofa', 
                'StoveBurner', 'TVStand', 'Toilet'}

    # Normalize the seen objects score to prevent the counter from going very high
    max_seen = max(seen_objects.values())
    normalized_scores = {obj: score/max_seen for obj, score in seen_objects.items()}
    # normalized_scores['None'] = 0
    # Aggregate scores for each sequence of actions based on normalized scores of seen objects
    seq_scores = []
    for index, seq in enumerate(list_of_list_of_actions):
        score = 0
        all_seen = True  # Assume initially that all objects in the sequence are seen
        for action in seq:
            obj = action[0]
            if task_type == 'pick_cool_then_place_in_recep' and obj == 'Fridge':
                score += 5
            elif task_type == 'pick_heat_then_place_in_recep' and obj == 'Microwave':
                score += 5
            elif task_type == 'pick_clean_then_place_in_recep' and obj == 'Faucet':
                score += 5
            elif task_type == 'look_at_obj_in_light' and 'Lamp' in obj:
                score += 5
            elif 'slice' in language_goal or 'cut' in language_goal or 'piece' in language_goal:
                if obj == 'Knife' or obj == 'ButterKnife':
                    score += 5

            # Merge 'Sink' and 'SinkBasin', as well as 'Bathtub' and 'BathtubBasin'
            if obj == 'SinkBasin':
                obj = 'Sink'
            elif obj == 'BathtubBasin':
                obj = 'Bathtub'
            elif obj == 'ButterKnife' or obj == 'Knife':
                obj = 'Knife'
            # Calculate score, penalize if object not seen
            if obj in normalized_scores: # Add normalized scores of seen objects
                score += normalized_scores[obj]
            else:  # Object not seen
                if not(obj.endswith('Sliced')) or (obj.endswith('Sliced') and obj[:-6] not in normalized_scores):  # Check if the object is a sliced version of a seen object
                    if (obj != 'Microwave'):
                        all_seen = False
                    if obj in receptacles and obj != 'Microwave': # Penalize unseen receptacles more
                        score -= 15
                    elif obj != 'Microwave': # Penalize unseen except microwave
                        score -= 0.5
        # If it's the first sequence and all objects are seen, add an extra 25 to the score
        if index == 0 and all_seen:
            score += 25
        seq_scores.append((score, seq))

    # Sort sequences based on their scores, highest first
    seq_scores.sort(reverse=True, key=lambda x: x[0])
    reordered_seqs = [seq for _, seq in seq_scores]

    return reordered_seqs


In [5]:
task = 'instruction goal is to put 2 dog decorations front to back on the edge of the right side of the table [SEP]turn around walk straight go to the brown couch walk between the couch and the table to the shelf on top of the couch [SEP]pick up the dog decoration from on top of the shelf [SEP]turn around and go to the table in front of the couch [SEP]put the dog on the edge of the right side of the table [SEP]walk to the right turn right at the end of the table walk toward the wall turn right to go to the table [SEP]pick up another dog decoration from on top of the table [SEP]turn around go to the left walk pass between the couch and the table and turn right to face the table again [SEP]put the dog decoration at the back of the other dog on the table '
seen_objects = {'Chair': 163, 'CoffeeTable': 260, 'Laptop': 157, 'CreditCard': 156, 'Watch': 77, 'Sofa': 243, 'SideTable': 182, 'DeskLamp': 59, 'Pillow': 106, 'Shelf': 260, 'WateringCan': 14, 'KeyChain': 146, 'Mirror': 33, 'FloorLamp': 38, 'GarbageCan': 17, 'Window': 35, 'Vase': 152, 'Statue': 92, 'RemoteControl': 70, 'Box': 69, 'HousePlant': 66, 'Newspaper': 82, 'Television': 3, 'Painting': 33, 'Bowl': 21, 'Blinds': 36, 'ArmChair': 86, 'Drawer': 207, 'LightSwitch': 2}
list_of_list_of_actions = [[('Statue', 'PickupObject'), ('CoffeeTable', 'PutObject'),('Statue', 'PickupObject'), ('CoffeeTable', 'PutObject')], 
                           [('Vase', 'PickupObject'), ('CoffeeTable', 'PutObject'), ('Vase', 'PickupObject'), ('CoffeeTable', 'PutObject')], 
                           [('RemoteControl', 'PickupObject'), ('CoffeeTable', 'PutObject'), ('RemoteControl', 'PickupObject'), ('CoffeeTable', 'PutObject')], 
                           [('AlarmClock', 'PickupObject'), ('CoffeeTable', 'PutObject'), ('AlarmClock', 'PickupObject'), ('CoffeeTable', 'PutObject')], 
                           [('PepperShaker', 'PickupObject'), ('CoffeeTable', 'PutObject'), ('PepperShaker', 'PickupObject'), ('CoffeeTable', 'PutObject')]]

reordered_sequences = get_reordered_actionSeq(list_of_list_of_actions, seen_objects, 'pick_heat_then_place_in_recep', task)
filtered_actionSeq = [action for sublist in reordered_sequences[:1] for action in sublist] # take the first 3 seq actions
filtered_actionSeq

[('Statue', 'PickupObject'),
 ('CoffeeTable', 'PutObject'),
 ('Statue', 'PickupObject'),
 ('CoffeeTable', 'PutObject')]